<a href="https://colab.research.google.com/github/ilakesh/courtwatchNOLA/blob/gh-pages/cap_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NOLA Court Dockets Data Analysis

In [ ]:
# Importing tools used for web scraping and data analysis
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import lxml as l
import datetime
import re

In [ ]:
def text_extract(s):
  '''This extracts the text between fourth and fifth lines and creates a new entity in a second data frame with columns:
    1) mag id
    2) date
    3) text
    4) judge
    '''

### The main function which extracts all relevant information from each docket

In [ ]:
def extract(soup):
  # WORKING REGEX TO GET MAG NUMBER
  mag_num = re.search(r'MAG#: (\d{6})', soup).group(1)
  mag_num

  # Using split with new line as delimeter -- standard spacing on top half of docket
  defendant = soup.split("\n")[9].strip()
  defendant

  # Getting the date and judge for the proceeding
  # It is always two lines after the 4th equal sign line
  text = soup.split("\n")
  equalsignline = 0
  charges = []
  for (count,line) in enumerate(text):
    if line == "=" * 78:
      equalsignline +=1
    if equalsignline == 2:
      charges.append(text[count])
    elif equalsignline == 4:
      #print(mag_num, text[count+2])
      date, judge = re.sub('\*','',text[count+2]).split()
      break
  # Takes "charges" from above and extracts count, code, bond, and charge
 
  cnt = []
  code = []
  bond = []
  charge = []

  for i, line in enumerate(charges[3:-1]):
    #print("i and line:",i,line)
    if 'BOND:' in line:
      s = line.split()
      #print("even s:", s)
      bond_ind = s.index("BOND:")
      cnt.append(s[0])
      code.append(" ".join(s[1:bond_ind]))
      bond.append(s[bond_ind +1])
    elif defendant in line:
      pass
    else:
      s = line.split()
      charge.append(" ".join(s[:]))

  # Duplicating the data that is the same for each charge on docket
  mag_nums = [mag_num] * len(bond)
  defendants = [defendant] * len(bond)
  dates = [date] *len(bond)
  judges = [judge] * len(bond)

  #Create a dataframe using the extracted data
  docket = pd.DataFrame({"Mag Num": mag_nums,"Defendant": defendants, "Judge": judges, "Count": cnt, "Code": code,"Charge": charge, "Bond": bond, "Date": dates})
  return docket

### The following initializes the dataframe "dockets" and uses **get** request to scrape text from each docket.  Then the extract function is called and the returned dataframe is added to the "dockets" dataframe

In [ ]:
# The prefix of every url we are scraping from
base_url = 'https://www.opso.us/dcktmstr/555555.php?&domagn='

# A list corresponding to the magistrate numbers of each docket
#l = list(range(586900, 587000))
l = [586545]
soups = []
dockets = pd.DataFrame()
for num in l:
  try:
    r = requests.get(base_url + str(num))
    root = BeautifulSoup( r.content )
    soup = str(root.find('pre'))
    soup.__repr__()
    dockets = dockets.append(extract(soup), ignore_index = True)
  except:
    print("Docket number {} is not working".format(str(num)))
display(dockets)

,Mag Num,Defendant,Judge,Count,Code,Charge,Bond,Date
0,586900,"JORDAN, WAYNE M",SCHIAFFIN,1,RS 14 35.3,DOMESTIC ABUSE BATTERY,0.00,08/06/2020
1,586901,"GIBSON, JASON M",SCHIAFFIN,1,RS 14 62.3,UNAUTHORIZED ENTRY INHABITED DWELLING,"1,000.00",08/06/2020
2,586901,"GIBSON, JASON M",SCHIAFFIN,1,RS 14 56(A)(1),SIMPLE CRIMINAL DAMAGE TO PROPERTY,"1,000.00",08/06/2020
3,586902,"NAULT, CHRISTOPHER",SCHIAFFIN,1,RS 14 35.3,DOMESTIC ABUSE BATTERY,0.00,08/06/2020
4,586903,"TROPEZ, DIANTE E",SCHIAFFIN,1,RS 14 67(B),THEFT,0.00,08/06/2020
...,...,...,...,...,...,...,...,...
251,586997,"BUSH, MICHAEL",MONTESSO,1,RS 14 68.4,UNAUTHORIZED USE OF MOTOR VEHICLE,0.00,08/11/2020
252,586998,"BASTISTE, LATRELL D Sr.",MONTESSO,1,RS 14 37.7 B (1),DOMESTIC ABUSE AGGRAVATED ASSAULT,0.00,08/11/2020
253,586998,"BASTISTE, LATRELL D Sr.",MONTESSO,1,RS 14 37.7 A,DOM ABUSE AGG ASS W/DANG WEAPON ON HHM,"1,000.00",08/11/2020
254,586999,"BERNIARD, VINCENT H",MONTESSO,1,RS 14 38,SIMPLE ASSAULT,0.00,08/11/2020


In [ ]:
## Initial cleaning of dataframe
# Removes "," from the bond value and typecasts bond to numeric
dockets["Bond"] = dockets["Bond"].str.replace(',','')
dockets["Bond"] = pd.to_numeric(dockets["Bond"])

In [ ]:
'''
dockets.groupby(["Charge"])["Bond"].mean()
dockets.groupby(["Charge"])["Bond"].count()
dockets.groupby(["Charge"])["Bond"].describe()
display(dockets)
'''

In [ ]:
#dockets = pd.DataFrame({"Mag Num": mag_nums,"Defendant": defendants, "Judge": judges, "Count": cnt, "Code": code,"Charge": charge, "Bond": bond})

In [ ]:
#display(dockets)

In [ ]:
# # WORKING REGEX TO GET BOND
# bond = re.findall(r'BOND:\s*(.*)\n', soup)
# bond